In [1]:
import imageio
import matplotlib.pyplot as plt
import numpy as np

import pandas as pd

In [ ]:
!pip install imageio-ffmpeg opencv-python -q

In [3]:
# Video file path
video_path = 'data/yantar-230722-02-det.mp4'

# Open the video file as a context manager
with imageio.get_reader(video_path) as video:
    # Read the first frame
    first_frame = video.get_data(0)

# Display the first frame using matplotlib
plt.imshow(first_frame)
plt.show()

# Save the first frame as an image using Matplotlib
plt.imsave('test.png', first_frame)

OSError: Could not load meta information
=== stderr ===

ffmpeg version 4.2.2-static https://johnvansickle.com/ffmpeg/  Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 8 (Debian 8.3.0-6)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-debug --disable-ffplay --disable-indev=sndio --disable-outdev=sndio --cc=gcc --enable-fontconfig --enable-frei0r --enable-gnutls --enable-gmp --enable-libgme --enable-gray --enable-libaom --enable-libfribidi --enable-libass --enable-libvmaf --enable-libfreetype --enable-libmp3lame --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-librubberband --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libvorbis --enable-libopus --enable-libtheora --enable-libvidstab --enable-libvo-amrwbenc --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libdav1d --enable-libxvid --enable-libzvbi --enable-libzimg
  libavutil      56. 31.100 / 56. 31.100


60;152 <br>
47;724 <br>
1620;125 <br>
2139;668

In [4]:
track_df = pd.read_csv("data/yantar-230722-02_track.csv")
track_df.sample()

,version https://git-lfs.github.com/spec/v1
1,size 53932465


In [5]:
import cv2

# Open the video file
cap = cv2.VideoCapture(video_path)

# Check if the video file was successfully opened
if not cap.isOpened():
    print("Error: Could not open video file.")
    exit()

# Get the resolution (width and height) of the video
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Release the video capture object
cap.release()

# Print the resolution
print(f"Video Resolution: {width} x {height}")

ModuleNotFoundError: No module named 'cv2'

In [ ]:
track_df.head()

In [ ]:
def downscale_df(df, ratio=2):
    df = df.copy()
    
    df["x1"] = df["x1"] / 2
    df["x2"] = df["x2"] / 2
    df["y1"] = df["y1"] / 2
    df["y2"] = df["y2"] / 2

    return df

track_df = downscale_df(track_df)

In [ ]:
df =  track_df[track_df["frame"]==1]

# Display the first frame using matplotlib
plt.imshow(first_frame)

# Draw bounding boxes on the image
for index, row in df.iterrows():
    x1, y1, x2, y2 = row['x1'], row['y1'], row['x2'], row['y2']
    rect = patches.Rectangle((x1, y1), x2 - x1, y2 - y1, linewidth=2, edgecolor='blue', facecolor='none')
    plt.gca().add_patch(rect)

# Save the image with bounding boxes
plt.savefig('output_with_boxes.png')

# Show the image
plt.show()

In [ ]:
edges = [(60, 152), ( 47,724), ( 1620,125), ( 2139,668)]
edges

In [ ]:
pd.set_option('display.float_format', lambda x: '%.0f' % x)

track_df.describe()

In [ ]:
track_yolo['FrameSequences'][0]["FPS"]

In [ ]:
def projection(X, Y):
    imageWidth = 2048 # в данном примере зададим их константами
    imageHeight = 726
    
    def geoToList(latlon):
      return np.array((latlon['lat'], latlon['lng']))
      
    def listToGeo(latlon):
      return {'lat': latlon[0], 'lng': latlon[1] }

    # (A,B,C,D) - corners of a football field
    # (X,Y) - some pixel 
    def getGeoCoordinates(A, B, C, D, X, Y) -> Tuple[float, float]:    
      A, B, C, D = list(map(geoToList, [A, B, C, D]))    
      vBC = (C - B) / imageHeight    
      vAD = (D - A) / imageHeight    
      latlonPixel1 = vBC * (imageHeight - Y) + B    
      latlonPixel2 = vAD * (imageHeight - Y) + A    
      vM = (latlonPixel2 - latlonPixel1) / imageWidth    
      M = vM * X + latlonPixel1    
      return listToGeo(M)
